In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import matplotlib.pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [3]:
with open('data/tiny_shakespeare.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
len(text)

1115394

### Vocab

In [4]:
unique_symbols = sorted(list(set(text)))
vocab_size = len(unique_symbols)

print(''.join(unique_symbols))
print(f'{vocab_size=}')


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size=65


### Encoder/decoder

In [5]:
stoi = {s: i for i, s in enumerate(unique_symbols)}
itos = {i: s for s, i in stoi.items()}

encode = lambda s: [stoi[ch] for ch in s]  # symbols to tokens
decode = lambda l: ''.join([itos[i] for i in l])  # tokens to symbols

### Train/val split

In [6]:
data = torch.tensor(encode(text), dtype = torch.long)
n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]

print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


### Build a dataset

In [7]:
torch.manual_seed(1337)
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x_batch = torch.stack([data[i:i+block_size] for i in ix])
    y_batch = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

    return x_batch, y_batch

## Model

In [8]:
# Head class
n_embd = 32

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.head_size = head_size

        # non-learnable data (do not receive gradients or participate in optimization)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # (B, T, head_size)
        k = self.key(x) # (B, T, head_size)
        wei = (q @ k.transpose(-2, -1)) * self.head_size ** -0.5 # (B, T, T)

        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # T can be smaller than block_size!!!!
        wei = F.softmax(wei, dim=-1)

        v = self.value(x) # (B, T, head_size)
        out = wei @ v # (B, T, T) @ (B, T, head_size) = (B, T, head_size)
        return out

In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim = -1)

In [10]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

In [11]:
# model
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()

        # emb table for token in vocab
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

        # emb table for each position
        self.pos_embedding_table = nn.Embedding(block_size, n_embd)

        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.sa_head = MultiHeadAttention(4, n_embd//4)
        self.ffwd = FeedForward(n_embd)

    def forward(self, idx, targets = None):
        B, T = idx.shape

        # channels = n_embd = 32 (not vocab_size!) so we do not receive logits directly anymore
        token_emb = self.token_embedding_table(idx) # (B, T, C), where B-batch, T-time, C-channel
        pos_emb = self.pos_embedding_table(torch.arange(T, device = device)) # (T, C)
        x = token_emb + pos_emb # x holds not just token identities but their positions as well
        x = self.sa_head(x)
        x = self.ffwd(x)
        logits = self.lm_head(x) # (B, T, V), where V-vocab_size

        if targets == None:
            loss = None
        else:
            # logits (B, T, C)
            # targets (B, T)
            # but F.cross_entropy expects input of shape (batch, C, <all other dims>) ---> error
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(-1))
            
        return logits, loss


    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cut = idx[:, -block_size:] # take only last block_size tokens
            logits, _ = self.forward(idx_cut)
            logits = logits[:, -1, :] # take only last token to predict the next one
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples= 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

In [12]:
m = Decoder().to(device)

In [13]:
num_eval_batches = 100

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(num_eval_batches)
        for n_batch in range(num_eval_batches):
            x_batch, y_batch = get_batch(split)
            _, loss = m(x_batch, y_batch)
            losses[n_batch] = loss.item()
        out[split] = losses.mean().item()
    m.train()
    return out

In [14]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [15]:
batch_size = 32
max_iters = 10_000
for step in range(max_iters):
    x_batch, y_batch = get_batch('train')
    logits, loss = m(x_batch, y_batch)

    if step % 1000 == 0 or step == max_iters - 1:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()  


# Single SA head loss was ~2.38
# before adding FFNN layer: 2.2132

step 0: train loss 4.2032, val loss 4.2009
step 1000: train loss 2.4716, val loss 2.4772
step 2000: train loss 2.3505, val loss 2.3616
step 3000: train loss 2.2837, val loss 2.3051
step 4000: train loss 2.2573, val loss 2.2613
step 5000: train loss 2.1988, val loss 2.2556
step 6000: train loss 2.1975, val loss 2.2227
step 7000: train loss 2.1845, val loss 2.2125
step 8000: train loss 2.1785, val loss 2.2082
step 9000: train loss 2.1707, val loss 2.2066
step 9999: train loss 2.1340, val loss 2.2014


In [18]:
start_token = torch.zeros((1, 1), dtype= torch.int, device = device)
gen_text = m.generate(start_token, 100)
print(decode(gen_text[0].tolist()))


BRETET:
Wans!
Al lind teall thull oncelfy:
As thavest, why mour nou norfoodengive whouldy would thak
